<a href="https://colab.research.google.com/github/kmkolasinski/tensorflow-nanoGPT/blob/main/gpt_2_finetune_conll2003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Fri May  5 13:40:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!git clone https://github.com/kmkolasinski/tensorflow-nanoGPT.git
!cd tensorflow-nanoGPT && pip install -r requirements.txt && pip install .

fatal: destination path 'tensorflow-nanoGPT' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.9 MB/s eta 0:00:00
     ━━

In [9]:
!cd tensorflow-nanoGPT && git pull &&  pip install .

Already up to date.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/tensorflow-nanoGPT
  Preparing metadata (setup.py) ... done
  Created wheel for tf-nano-gpt: filename=tf_nano_gpt-0.1.0-py3-none-any.whl size=15205 sha256=b8f13220e661045aa5e1b619df3d8f79c36deb22f206191ca3fe61ad4568b1c1
  Stored in directory: /root/.cache/pip/wheels/5b/f2/91/1326ae5d7c87e9a08e865f2ba555e05e80fb5cfcdfd5cc76c1
Successfully built tf-nano-gpt


In [10]:
!cp -r tensorflow-nanoGPT/notebooks/tokenizer tokenizer

In [11]:
%load_ext autoreload
%autoreload 2

In [12]:
import tensorflow as tf
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Dataset preparation

In [13]:
from datasets import load_dataset
dataset = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
NER_TAGS = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-MISC": 7,
    "I-MISC": 8,
}
NER_TAGS_INV = {v: k for k, v in NER_TAGS.items()}

In [15]:
dataset["train"][100]

{'id': '100',
 'tokens': ['Rabinovich',
  'is',
  'winding',
  'up',
  'his',
  'term',
  'as',
  'ambassador',
  '.'],
 'pos_tags': [21, 42, 39, 33, 29, 21, 15, 21, 7],
 'chunk_tags': [11, 21, 22, 15, 11, 12, 13, 11, 0],
 'ner_tags': [1, 0, 0, 0, 0, 0, 0, 0, 0]}

In [16]:
from tqdm import tqdm

train_data = []
for data in tqdm(dataset["train"]):
    tags = [NER_TAGS_INV[tag] for tag in data["ner_tags"]]
    train_data.append((data["tokens"], tags))




  0%|          | 0/14041 [00:00<?, ?it/s]


  5%|▍         | 667/14041 [00:00<00:02, 6660.57it/s]


 10%|▉         | 1334/14041 [00:00<00:01, 6504.02it/s]


 14%|█▍        | 1985/14041 [00:00<00:01, 6367.86it/s]


 19%|█▉        | 2703/14041 [00:00<00:01, 6677.72it/s]


 24%|██▍       | 3372/14041 [00:00<00:01, 6347.93it/s]


 29%|██▉       | 4092/14041 [00:00<00:01, 6624.65it/s]


 34%|███▍      | 4758/14041 [00:00<00:01, 6401.39it/s]


 39%|███▉      | 5481/14041 [00:00<00:01, 6650.89it/s]


 44%|████▍     | 6150/14041 [00:00<00:01, 6614.73it/s]


 49%|████▉     | 6865/14041 [00:01<00:01, 6773.86it/s]


 54%|█████▎    | 7545/14041 [00:01<00:00, 6640.30it/s]


 58%|█████▊    | 8211/14041 [00:01<00:01, 3730.05it/s]


 63%|██████▎   | 8877/14041 [00:01<00:01, 4290.46it/s]


 67%|██████▋   | 9441/14041 [00:01<00:01, 4560.41it/s]


 71%|███████▏  | 10031/14041 [00:01<00:00, 4871.88it/s]


 76%|███████▌  | 10652/14041 [00:01<00:00, 5204.74it/s]


 80%|████████  | 11238/14041 [00:02<00:0

In [17]:
validation_data = []
for data in tqdm(dataset["validation"]):
    tags = [NER_TAGS_INV[tag] for tag in data["ner_tags"]]
    validation_data.append((data["tokens"], tags))

100%|██████████| 3250/3250 [00:00<00:00, 11473.92it/s]


In [18]:
import numpy as np
np.quantile([len(data[0]) for data in train_data], 0.95)

37.0

In [19]:
def prepare_data(item):
    targets_data = []
    current_ner = []

    for token, tag_name in zip(*item):

        if tag_name.startswith("B-"):
            if len(current_ner) == 0:
                current_ner.append((token, tag_name[2:]))
                continue
            else:
                targets_data.append(current_ner)
                current_ner = []
                current_ner.append((token, tag_name[2:]))
                continue

        if tag_name.startswith("I-") and len(current_ner) > 0:
            current_ner.append((token, tag_name[2:]))
            continue

        if len(current_ner) > 0:
            targets_data.append(current_ner)
            current_ner = []

    if len(current_ner) > 0:
        targets_data.append(current_ner)


    context = " ".join(item[0])
    
    target_text = ""
    for tokens in targets_data:
        words = " ".join([word for word, tag in tokens])
        target_text += f"{words}//{tokens[0][1]}\n"

    return context, target_text


item = train_data[5]
for token, tag_name in zip(*item):
    print(f"{token:15}{tag_name}")

context_text, target_text = prepare_data(item)

print(f"prompt:{context_text}")
print(f"target:{target_text}")

"              O
We             O
do             O
n't            O
support        O
any            O
such           O
recommendation O
because        O
we             O
do             O
n't            O
see            O
any            O
grounds        O
for            O
it             O
,              O
"              O
the            O
Commission     B-ORG
's             O
chief          O
spokesman      O
Nikolaus       B-PER
van            I-PER
der            I-PER
Pas            I-PER
told           O
a              O
news           O
briefing       O
.              O
prompt:" We do n't support any such recommendation because we do n't see any grounds for it , " the Commission 's chief spokesman Nikolaus van der Pas told a news briefing .
target:Commission//ORG
Nikolaus van der Pas//PER



In [20]:
prepared_train_data = [prepare_data(item) for item in tqdm(train_data)]
prepared_validation_data = [prepare_data(item) for item in tqdm(validation_data)]

100%|██████████| 3250/3250 [00:00<00:00, 52020.64it/s]


# Build tf.data.Dataset iterators

In [21]:
from tf_nano_gpt.model import GPT2Tokenizer

sequence_length = 256
context_length =  sequence_length // 2
tokenizer = GPT2Tokenizer()

In [22]:
def prepare_lm_targets(context, target):
    context_tokens, target_tokens = tokenizer.tokenize_sample(context, target)
    context_tokens = tokenizer.pad_or_slice(context_tokens, context_length)
    target_tokens = tokenizer.pad_or_slice(target_tokens, context_length + 1)

    x = tf.concat([context_tokens, target_tokens[:-1]], 0)
    y = tf.concat([context_tokens, target_tokens[1:]], 0)

    # simple mask to remove context from the loss computation as well as pad tokens
    mask = tf.cast(tf.abs(x - y) > 0, tf.int32)
    targets_ids = tf.stack([y, mask], 1)

    return {"inputs_ids": x, "targets_ids": targets_ids}, targets_ids

In [23]:
def build_dataset_iterator(prepared_data, is_training: bool, batch_size: int) -> tf.data.Dataset:
    dataset = tf.data.Dataset.from_tensor_slices(prepared_data)
    dataset = dataset.map(
        lambda x: prepare_lm_targets(x[0], x[1]), num_parallel_calls=tf.data.AUTOTUNE
    )

    if is_training:
        dataset = dataset.repeat(-1)
        dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [24]:
batch_size = 8
train_dataset = build_dataset_iterator(prepared_train_data, is_training=True, batch_size=batch_size)
train_dataset

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<PrefetchDataset element_spec=({'inputs_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name=None), 'targets_ids': TensorSpec(shape=(None, 256, 2), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 256, 2), dtype=tf.int32, name=None))>

In [25]:
validation_dataset = build_dataset_iterator(prepared_validation_data, is_training=False, batch_size=batch_size)
validation_dataset

<PrefetchDataset element_spec=({'inputs_ids': TensorSpec(shape=(None, 256), dtype=tf.int32, name=None), 'targets_ids': TensorSpec(shape=(None, 256, 2), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 256, 2), dtype=tf.int32, name=None))>

In [26]:
for x, y in train_dataset:
    break

x = x["inputs_ids"]
pad_str = tokenizer.pad_token_str
print(tokenizer.detokenize(x[0, :context_length]).numpy().decode("utf-8").replace(pad_str, ""))
print(tokenizer.detokenize(y[0, context_length:, 0]).numpy().decode("utf-8").replace(pad_str, ""))

WESTERN DIVISION
WESTERN DIVISION//MISC
<|endoftext|>


# Build model

In [27]:
from transformers import TFGPT2Model
from tf_nano_gpt.model import freeze_embeddings, freeze_layers
from tf_nano_gpt.metrics import masked_lm_loss, masked_accuracy

base_model = TFGPT2Model.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


In [28]:
freeze_embeddings(base_model)
freeze_layers(base_model, num_blocks_to_freeze=8, use_lora=False)

In [29]:
def model_predict_fn(
    inputs_ids: tf.Tensor,
    past_key_values: tf.Tensor = None,
    position_ids: tf.Tensor = None,
):
    encoded_input = {
        "input_ids": inputs_ids,
        "attention_mask": tf.ones_like(inputs_ids),
        "past_key_values": past_key_values,
        "position_ids": position_ids,
    }

    output = base_model(encoded_input)
    last_hidden_state = output.last_hidden_state
    past_key_values = output.past_key_values

    last_hidden_state = tf.keras.layers.Dropout(0.2)(last_hidden_state)

    logits = base_model.transformer.wte(last_hidden_state, mode="linear")
    return logits, past_key_values


inputs_ids = tf.keras.layers.Input(
    shape=(sequence_length,), dtype=tf.int32, name="inputs_ids"
)
targets_ids = tf.keras.layers.Input(
    shape=(sequence_length, 2), dtype=tf.int32, name="targets_ids"
)

logits, _ = model_predict_fn(inputs_ids)

loss_value = masked_lm_loss(targets_ids, logits)
accuracy_value = masked_accuracy(targets_ids, tf.argmax(logits, -1))

train_model = tf.keras.Model(inputs=[inputs_ids, targets_ids], outputs=logits)

train_model.add_loss(loss_value)
train_model.add_metric(accuracy_value, name="accuracy")

train_model.summary(180)

Model: "model"
____________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                              Output Shape                            Param #              Connected to                                                
 inputs_ids (InputLayer)                                   [(None, 256)]                           0                    []                                                          
                                                                                                                                                                                    
 tf.ones_like (TFOpLambda)                                 (None, 256)                             0                    ['inputs_ids[0][0]']                                        
                                                                                

In [30]:
inference_model = tf.keras.Model(inputs=inputs_ids, outputs=logits)

# Training model

In [31]:
epochs = 10
steps_per_epoch = 1000
save_dir = "models/test-gpt-2-model-v1"


In [33]:
decay_steps = steps_per_epoch * epochs
validation_steps = validation_dataset.cardinality().numpy() // batch_size
validation_steps

50

In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
train_model.compile(optimizer, jit_compile=True)

In [35]:
from pathlib import Path

lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(0.0001, epochs, alpha=0.01)

def lr_schedule(epoch):
  learning_rate = lr_scheduler(epoch)
  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate


callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=Path(save_dir) / "logs",
        histogram_freq=0,
        embeddings_freq=0,
        update_freq="epoch",
        write_steps_per_second=True,
        profile_batch=(200, 250),
        write_graph=False,
    ),
    tf.keras.callbacks.LearningRateScheduler(lr_schedule),
    tf.keras.callbacks.ModelCheckpoint(
        Path(save_dir) / "model",
        monitor = "val_accuracy",
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
        mode = "auto",
    ),
]

In [36]:
# train_model.load_weights(Path(save_dir) / "model")

In [37]:
for x, y in train_dataset:
    break

y_pred = train_model(x)
y_pred.shape

TensorShape([8, 256, 50257])

In [38]:
train_model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps,
    verbose=1,
    callbacks=callbacks,
    epochs=epochs,
)

Epoch 1/10
1000/1000 [==============================] - ETA: 0s - loss: 0.0390 - accuracy: 0.8351
Epoch 1: val_accuracy improved from -inf to 0.93714, saving model to models/test-gpt-2-model-v1/model
1000/1000 [==============================] - 161s 105ms/step - loss: 0.0390 - accuracy: 0.8351 - val_loss: 0.0155 - val_accuracy: 0.9371 - lr: 1.0000e-04
Epoch 2/10
1000/1000 [==============================] - ETA: 0s - loss: 0.0243 - accuracy: 0.8921
Epoch 2: val_accuracy improved from 0.93714 to 0.95425, saving model to models/test-gpt-2-model-v1/model
1000/1000 [==============================] - 91s 91ms/step - loss: 0.0243 - accuracy: 0.8921 - val_loss: 0.0107 - val_accuracy: 0.9542 - lr: 9.7577e-05
Epoch 3/10
1000/1000 [==============================] - ETA: 0s - loss: 0.0174 - accuracy: 0.9194
Epoch 3: val_accuracy improved from 0.95425 to 0.95496, saving model to models/test-gpt-2-model-v1/model
1000/1000 [==============================] - 91s 91ms/step - loss: 0.0174 - accuracy: 0.

# Evaluate model

In [39]:
validation_dataset_iter = iter(validation_dataset)

In [40]:
def greedy_predict_next_token(inputs_ids: tf.Tensor) -> tf.Tensor:
    
    current_index = tf.reduce_sum(tf.cast(inputs_ids > 0, tf.int32), -1) - 1
    num_sentences, maxlen = tf.shape(inputs_ids)[0], tf.shape(inputs_ids)[1]

    y = inference_model(inputs_ids)
    logits = tf.gather(y, current_index, batch_dims=1)
    sampled_indices = tf.argmax(logits, axis=-1, output_type=tf.int32)

    current_index = tf.minimum(current_index + 1, maxlen - 1)
    scatter_indices = tf.stack([tf.range(num_sentences), current_index], axis=1)

    inputs_ids = (
        tf.scatter_nd(scatter_indices, sampled_indices, shape=(num_sentences, maxlen))
        + inputs_ids
    )
    return inputs_ids

In [41]:
x, y = next(validation_dataset_iter)
x = x['inputs_ids']

In [42]:
idx = 0
x_test = tf.concat([x[:, :context_length + 1], tf.zeros_like(x)[:, context_length + 1:]], -1)
context, target = tokenizer.detokenize(x)[idx].numpy().decode().split(tokenizer.start_token_str)
target_text = tokenizer.detokenize(y[:, context_length:, 0])[idx].numpy().decode()

print(context.replace(tokenizer.pad_token_str, ""))
print(target_text.replace(tokenizer.pad_token_str, ""))

CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .
LEICESTERSHIRE//ORG
<|endoftext|>


In [43]:
stop_token = tokenizer.stop_token_id
for i in range(context_length):
    x_test = greedy_predict_next_token(x_test)
    all_complete = tf.shape(tf.unique(tf.where(x_test[:, context_length:] == stop_token)[:, 0]).y) == batch_size
    if bool(all_complete[0].numpy()):
        break

In [44]:
result = tokenizer.detokenize(x_test[:, context_length + 1:])[idx].numpy().decode().replace("!", "")
print(result)

LEICESTERSHIRE//LOC
<|endoftext|>::PER
<|endoftext|>::PER
<|endoftext|>::PER
<|endoftext|>::PER
<|endoftext|>::


# Sampling using keras_nlp functions

In [45]:
import keras_nlp


@tf.function(input_signature=[tf.TensorSpec(shape=(None, None), dtype=tf.int32)])
def token_probability_fn(inputs):

    input_len = tf.shape(inputs)[1]

    inputs = tf.map_fn(
        lambda _: tokenizer.pad_or_slice(_, sequence_length), inputs
    )

    y = inference_model(inputs)
    return y[:, input_len - 1, :]

In [46]:
prompt = x[idx, : context_length + 1][None, :]

predicted_tokens = keras_nlp.utils.top_k_search(
    token_probability_fn,
    prompt,
    max_length=sequence_length,
    end_token_id=tokenizer.stop_token_id,
    from_logits=True,
    k=5,
)

if len(predicted_tokens.shape) == 1:
    predicted_tokens = predicted_tokens[None, :]
predicted_tokens.shape

TensorShape([1, 256])

In [47]:
result = tokenizer.detokenize(predicted_tokens[:, context_length + 1:])[0].numpy().decode().replace("!", "")
print(result)

LEICESTERSHIRE//LOC
<|endoftext|>


# Export model - base method

In [48]:
class GPT2Exporter(tf.Module):
    def __init__(
        self, model: tf.keras.Model, tokenizer: GPT2Tokenizer, jit_compile: bool = None
    ):
        super(GPT2Exporter, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.jit_compile = jit_compile
        self.predict_next_token_fn = greedy_predict_next_token

        if jit_compile:
            self.predict_next_token_fn = tf.function(
                greedy_predict_next_token, jit_compile=True
            )

    def prepare_inputs(self, text: str):
        input_ids = self.tokenizer.tokenize(text)
        input_ids = self.tokenizer.pad_or_slice(input_ids, context_length)
        input_ids = self.tokenizer.pad_or_slice(
            tf.concat([input_ids, [self.tokenizer.start_token_id]], 0),
            sequence_length,
            pad_value=0,
        )

        input_ids = tf.reshape(input_ids, [1, -1])

        return input_ids

    @tf.function(input_signature=[tf.TensorSpec([], tf.string)])
    def __call__(self, text):
        input_ids = self.prepare_inputs(text)

        i = tf.constant(0)
        while i < context_length:
            i += 1
            input_ids = self.predict_next_token_fn(input_ids)

            completed = tf.reduce_any(input_ids == self.tokenizer.stop_token_id)

            if completed:
                break

        prediction = self.tokenizer.detokenize(input_ids[:, : context_length + i - 1])
        prediction = tf.strings.split(prediction, self.tokenizer.start_token_str)[0, 1]

        return {"outputs": prediction}


gpt2_predictor = GPT2Exporter(inference_model, tokenizer)
gpt2_predictor_jit = GPT2Exporter(inference_model, tokenizer, jit_compile=True)

In [49]:
prompt = "Headingley is a suburb of Leeds, West Yorkshire, England, approximately two miles out of the city centre, to the north west along the A660 road. Headingley is the location of the Beckett Park campus of Leeds Beckett University and Headingley Stadium."

prediction = gpt2_predictor(prompt)
prediction['outputs'].numpy().decode().split("\n")

['Headingley//LOC',
 'Leeds//LOC',
 'West Yorkshire//LOC',
 'Headingley//LOC',
 'Leeds University//ORG',
 'Headingley Stadium//LOC']

In [50]:
prediction = gpt2_predictor_jit(prompt)
prediction['outputs'].numpy().decode().split("\n")

['Headingley//LOC',
 'Leeds//LOC',
 'West Yorkshire//LOC',
 'Headingley//LOC',
 'Leeds University//ORG',
 'Headingley Stadium//LOC']

In [51]:
%timeit gpt2_predictor(prompt)

579 ms ± 38.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%timeit gpt2_predictor_jit(prompt)

369 ms ± 4.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
tf.saved_model.save(gpt2_predictor, Path(save_dir) / "exported-models/gpt2-ner/1/")
tf.saved_model.save(gpt2_predictor_jit, Path(save_dir) / "exported-models/gpt2-ner-jit/1/")

# Export model with cached keys and queries
This is applicable only for GPT2 small which has 12 blocks

In [53]:
def sample_argmax(logits):
    return tf.argmax(logits, axis=-1, output_type=tf.int32)


class CachedGPT2Exporter(GPT2Exporter):
    def __init__(
        self, model: tf.keras.Model, tokenizer: GPT2Tokenizer, jit_compile: bool = False
    ):
        super(CachedGPT2Exporter, self).__init__(model, tokenizer, jit_compile=False)
        self.predict_next_token_fn = model_predict_fn
        if jit_compile:
            self.predict_next_token_fn = tf.function(model_predict_fn, jit_compile=True)

    @tf.function(input_signature=[tf.TensorSpec([], tf.string)])
    def __call__(self, text):
        input_ids = self.prepare_inputs(text)
        input_ids = input_ids[:, : context_length + 1]

        logits, past_key_values = self.predict_next_token_fn(input_ids)
        new_input_ids = tf.expand_dims(sample_argmax(logits[:, -1, :]), axis=-1)

        states = tf.TensorArray(tf.int32, size=context_length)
        states = states.write(0, new_input_ids)

        # for autograph to compile this function we need to specify each variable explicitly
        # since set_loop_options does not woth with python lists, that's why this function
        # will work only with GPT2 small model
        kv0 = past_key_values[0]
        kv1 = past_key_values[1]
        kv2 = past_key_values[2]
        kv3 = past_key_values[3]
        kv4 = past_key_values[4]
        kv5 = past_key_values[5]
        kv6 = past_key_values[6]
        kv7 = past_key_values[7]
        kv8 = past_key_values[8]
        kv9 = past_key_values[9]
        kv10 = past_key_values[10]
        kv11 = past_key_values[11]

        for i in tf.range(context_length - 1):
            tf.autograph.experimental.set_loop_options(
                shape_invariants=[
                    (kv0, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv1, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv2, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv3, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv4, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv5, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv6, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv7, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv8, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv9, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv10, tf.TensorShape([2, 1, 12, None, 64])),
                    (kv11, tf.TensorShape([2, 1, 12, None, 64])),
                ]
            )
            past_key_values = [
                kv0,
                kv1,
                kv2,
                kv3,
                kv4,
                kv5,
                kv6,
                kv7,
                kv8,
                kv9,
                kv10,
                kv11,
            ]

            past_length = i + context_length - 1
            position_ids = tf.expand_dims(
                tf.range(past_length, past_length + 1), axis=0
            )

            logits, new_past_key_values = self.predict_next_token_fn(
                inputs_ids=new_input_ids,
                past_key_values=past_key_values,
                position_ids=position_ids,
            )
            new_input_ids = tf.expand_dims(sample_argmax(logits[:, -1, :]), axis=-1)
            states = states.write(i + 1, new_input_ids)

            kv0 = new_past_key_values[0]
            kv1 = new_past_key_values[1]
            kv2 = new_past_key_values[2]
            kv3 = new_past_key_values[3]
            kv4 = new_past_key_values[4]
            kv5 = new_past_key_values[5]
            kv6 = new_past_key_values[6]
            kv7 = new_past_key_values[7]
            kv8 = new_past_key_values[8]
            kv9 = new_past_key_values[9]
            kv10 = new_past_key_values[10]
            kv11 = new_past_key_values[11]

            completed = tf.reduce_any(new_input_ids == self.tokenizer.stop_token_id)

            if completed:
                break

        input_ids = tf.reshape(states.stack(), [1, -1])[:, :i]

        prediction = self.tokenizer.detokenize(input_ids)

        return {"outputs": prediction[0]}


gpt2_cached_predictor = CachedGPT2Exporter(
    inference_model, tokenizer, jit_compile=False
)
gpt2_cached_predictor_jit = CachedGPT2Exporter(
    inference_model, tokenizer, jit_compile=True
)

In [54]:
prediction = gpt2_cached_predictor(prompt)
prediction['outputs'].numpy().decode().replace("!", "").split("\n")

['Headingley//LOC',
 'Leeds//LOC',
 'West Yorkshire//LOC',
 'Headingley//LOC',
 'Leeds University//ORG',
 'Headingley Stadium//LOC']

In [55]:
# this will take a while for google colab
prediction = gpt2_cached_predictor_jit(tf.constant(prompt))
prediction['outputs'].numpy().decode().replace("!", "").split("\n")

['Headingley//LOC',
 'Leeds//LOC',
 'West Yorkshire//LOC',
 'Headingley//LOC',
 'Leeds University//ORG',
 'Headingley Stadium//LOC']

In [58]:
%timeit gpt2_cached_predictor(tf.constant(prompt))

688 ms ± 125 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit gpt2_cached_predictor_jit(tf.constant(prompt))

245 ms ± 7.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
tf.saved_model.save(gpt2_cached_predictor, Path(save_dir) / "exported-models/gpt2-ner-cached/1/")
tf.saved_model.save(gpt2_cached_predictor_jit, Path(save_dir) / "exported-models/gpt2-ner-cached-jit/1/")

In [ ]:
loaded_model = tf.saved_model.load("models/test-gpt-2-model/exported-models/gpt2-ner-cached-jit/1")
loaded_model.signatures

# Test tensorflow Serving

In [ ]:
from pathlib import Path


cwd = Path.cwd()
save_dir = "models/test-gpt-2-model"

run_serving_cmd = f"docker run -p 8501:8501 --rm --gpus all --name tfserving_models --mount type=bind,source={cwd}/{save_dir}/exported-models/gpt2-ner,target=/models/model -e MODEL_NAME=model -t tensorflow/serving:2.11.1-gpu"
print(run_serving_cmd)

In [ ]:
import requests
import json

context = "Headingley is a suburb of Leeds, West Yorkshire, England, approximately two miles out of the city centre, to the north west along the A660 road. Headingley is the location of the Beckett Park campus of Leeds Beckett University and Headingley Stadium."

prediction_url = "http://localhost:8501/v1/models/model:predict"
post_data = {"inputs": {"text": context}}
response = requests.post(prediction_url, data=json.dumps(post_data))
prediction = response.json()["outputs"]
prediction.split("\n")